In [ ]:
from abc import ABC, abstractmethod
import os
import signal
import sys

from IPython.display import display, clear_output
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
import torch

sys.path.append('/home/apankov/dev.vectorization')
from vectran.renderers.cairo import render as original_render, render_with_skeleton as original_render_with_skeleton, PT_LINE


os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

## No NaN Adam

In [ ]:
import math
import torch


class NonanAdam(torch.optim.Adam):
    def step(self, closure=None):
        """Performs a single optimization step.

        Arguments:
            closure (callable, optional): A closure that reevaluates the model
                and returns the loss.
        """
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data
                grad[~torch.isfinite(grad)] = 0 ### <---
                if grad.is_sparse:
                    raise RuntimeError('Adam does not support sparse gradients, please consider SparseAdam instead')
                amsgrad = group['amsgrad']

                state = self.state[p]

                # State initialization
                if len(state) == 0:
                    state['step'] = 0
                    # Exponential moving average of gradient values
                    state['exp_avg'] = torch.zeros_like(p.data)
                    # Exponential moving average of squared gradient values
                    state['exp_avg_sq'] = torch.zeros_like(p.data)
                    if amsgrad:
                        # Maintains max of all exp. moving avg. of sq. grad. values
                        state['max_exp_avg_sq'] = torch.zeros_like(p.data)

                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                if amsgrad:
                    max_exp_avg_sq = state['max_exp_avg_sq']
                beta1, beta2 = group['betas']

                state['step'] += 1

                if group['weight_decay'] != 0:
                    grad.add_(group['weight_decay'], p.data)

                # Decay the first and second moment running average coefficient
                exp_avg.mul_(beta1).add_(1 - beta1, grad)
                exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)
                if amsgrad:
                    # Maintains the maximum of all 2nd moment running avg. till now
                    torch.max(max_exp_avg_sq, exp_avg_sq, out=max_exp_avg_sq)
                    # Use the max. for normalizing running avg. of gradient
                    denom = max_exp_avg_sq.sqrt().add_(group['eps'])
                else:
                    denom = exp_avg_sq.sqrt().add_(group['eps'])

                bias_correction1 = 1 - beta1 ** state['step']
                bias_correction2 = 1 - beta2 ** state['step']
                step_size = group['lr'] * math.sqrt(bias_correction2) / bias_correction1
                
                assert np.isfinite(step_size)
                assert torch.all(torch.isfinite(exp_avg))
                assert torch.all(torch.isfinite(denom))

                p.data.addcdiv_(-step_size, exp_avg, denom)

        return loss

## Parameters and utils

In [ ]:
device = torch.device('cuda')
dtype = torch.float32
division_epsilon = 1e-12
min_linear = 2**-8
empty_charge = 0
visibility_padding = 2
elementary_halfwidth = 1 / 2

In [ ]:
def render(data, dimensions, data_representation):
    return original_render(data, dimensions, data_representation, linecaps='butt', linejoin='miter')


def render_skeleton(data, dimensions, data_representation, padding=0, scaling=4):
    w, h = dimensions
    data[..., [0, 2]] = np.clip(data[..., [0, 2]], .5, w-.5)
    data[..., [1, 3]] = np.clip(data[..., [1, 3]], .5, h-.5)
    data[..., :4] += padding
    data = data * [scaling, scaling, scaling, scaling, 1]
    data[..., :4][data[..., -1] < 1/4] = -100
    data[..., -1] = 0
    return original_render_with_skeleton({PT_LINE: data}, [(w + padding*2)*scaling, (h + padding*2)*scaling], data_representation, linecaps='butt', linejoin='miter', line_color=(0,0,0), line_width=2)
    

def get_random_line(h, w, max_width=None):
    if max_width is None:
        max_width = max(h,w) // 10
    line = np.float32(np.random.random(5) * [w, h, w, h, max_width])
    line[..., -1] += 1
    return line

## Differentiable rasterizer with supersampling

In [ ]:
class SupersamplingStrategy(ABC):
    @abstractmethod
    def supersample(self, raster_coordinates):
        """Generate coordinates of supersamples for given coordinates of rasters."""
        pass

    def subsample(self, supersamples):
        """Average supersamples to get the actual raster"""
        pass
    
    def reorder_supersamples(self, supersamples):
        """Reorder supersamples, generated for C ordered (row major) raster_coordinates, for being in C order.
        
        Parameters
        ----------
        supersamples : torch.Tensor
            Tensor of shape [**, h, w]
        """
        pass


class RegularSupersampling(SupersamplingStrategy):
    def __init__(self, supersamples_per_dimension=4):
        self.supersamples_per_dimension = supersamples_per_dimension
    
    def supersample(self, raster_coordinates):
        """Generate coordinates of supersamples for given coordinates of rasters."""
        n = self.supersamples_per_dimension
        rasters_n = raster_coordinates.shape[1]
        n2 = n ** 2
        supersample_coordinates = torch.empty([2, rasters_n * n2], dtype=raster_coordinates.dtype, device=raster_coordinates.device)
        for i in range(n):
            xs = raster_coordinates[0] + (i / n - (n-1) / (n * 2))
            ys = raster_coordinates[1] + (i / n - (n-1) / (n * 2))
            for j in range(n):
                supersample_coordinates[0, i + n * j :: n2] = xs
                supersample_coordinates[1, i * n + j :: n2] = ys
        del xs, ys
        return supersample_coordinates

    def subsample(self, supersamples, dtype=torch.float32):
        """Average supersamples to get the actual raster"""
        return torch.nn.functional.avg_pool1d(supersamples.type(dtype), self.supersamples_per_dimension**2)

    def reorder_supersamples(self, supersamples):
        """Reorder supersamples, generated for C ordered (row major) raster_coordinates, for being in C order."""
        shape = supersamples.shape
        other_dims = shape[:-2]
        H = W = self.supersamples_per_dimension
        hH, wW = shape[-2:]
        assert ( hH % H == 0 ) and ( wW % W == 0 )
        h = hH // H
        w = wW // W
        supersamples = supersamples.reshape(*other_dims, h, w, H, W).transpose(-3, -2)
        return supersamples.reshape(shape)


class RGSSSupersampling(SupersamplingStrategy):
    """Only 4xRGSS by now."""

    def supersample(self, raster_coordinates):
        """Generate coordinates of supersamples for given coordinates of rasters."""
        rasters_n = raster_coordinates.shape[1]
        supersample_coordinates = torch.empty([2, rasters_n * 4], dtype=raster_coordinates.dtype, device=raster_coordinates.device)
        supersample_coordinates[0, 0::4] = raster_coordinates[0] + 1/8
        supersample_coordinates[1, 0::4] = raster_coordinates[1] - 3/8
        supersample_coordinates[0, 1::4] = raster_coordinates[0] - 1/8
        supersample_coordinates[1, 1::4] = raster_coordinates[1] - 1/8
        supersample_coordinates[0, 2::4] = raster_coordinates[0] + 3/8
        supersample_coordinates[1, 2::4] = raster_coordinates[1] + 1/8
        supersample_coordinates[0, 3::4] = raster_coordinates[0] - 1/8
        supersample_coordinates[1, 3::4] = raster_coordinates[1] + 3/8
        return supersample_coordinates

    def subsample(self, supersamples, dtype=torch.float32):
        """Average supersamples to get the actual raster"""
        return torch.nn.functional.avg_pool1d(supersamples.type(dtype), 4)
    
    def reorder_supersamples(self, supersamples):
        """Reorder supersamples, generated for C ordered (row major) raster_coordinates, for being in C order."""
        raise NotImplementedError

In [ ]:
def render_lines(x1, y1, x2, y2, width, sample_coordinates, samples=None, linecaps='butt',
                 dtype=torch.float16, requires_grad=False, sparse=False):
    """...
    Requires 34 bytes of GPU memory per patch per line per sample for half precision computations (default),
             45 for single precision computations, and 65 for double presision computations,
     i.e 2.7/3.5/5 GB to render 128 patches of 10 lines in patches of size 64x64 with 4x4 supersampling.
    
    Parameters
    ----------
        samples : torch.BoolTensor
    """
    # assert linecaps == 'butt', 'Others are not implemented by now (but easy to implement)'
    # assert not requires_grad, 'Not implemented'
    batch_size, lines_n = x1.shape
    rasters_n = sample_coordinates.shape[1]
    
    with torch.set_grad_enabled(requires_grad):
        l_dir = torch.empty(2, batch_size, lines_n, dtype=dtype, device=x1.device)
        l_dir[0] = x2.type(dtype)
        l_dir[0] -= x1.type(dtype)
        l_dir[1] = y2.type(dtype)
        l_dir[1] -= y1.type(dtype)
        length = torch.norm(l_dir, dim=0)
        l_dir /= (length + division_epsilon)

        R = torch.empty(2, batch_size, lines_n, rasters_n, dtype=l_dir.dtype, device=l_dir.device)
        R[0] = sample_coordinates[0]
        R[0] -= x1[..., None].type(R.dtype)
        R[1] = sample_coordinates[1]
        R[1] -= y1[..., None].type(R.dtype)
        
        proj = l_dir[0, ..., None] * R[0]
        proj += l_dir[1, ..., None] * R[1]
        
        shaded = proj <= length[..., None]
        shaded &= proj >= 0
        del length
        
        dist = proj
        dist[:] = l_dir[0, ..., None] * R[1]
        dist -= l_dir[1, ..., None] * R[0]
        dist.abs_()
        
        half_width = (width / 2).type(dtype)
        shaded &= dist <= half_width[..., None]
        del dist, proj, R, l_dir, half_width
        
        if sparse:
            ids = torch.nonzero(shaded).t().contiguous()
            del shaded
            if samples is None:
                samples = torch.sparse.ByteTensor(ids, torch.ones(1, dtype=torch.uint8, device=width.device).expand(ids.shape[1]))
                del ids
                return samples
            else:
                pass # TODO
        else:
            if samples is None:
                return shaded
            else:
                samples += shaded
                del shaded

### Define raster grid

In [ ]:
h, w = 64, 64
padding = 3
padded_h = h + padding*2
padded_w = w + padding*2
supersampling_strategy = RegularSupersampling(4)
pixel_center_coodinates_are_integer = False

raster_coordinates = torch.meshgrid(torch.arange(-padding, h+padding, dtype=dtype), torch.arange(-padding, w+padding, dtype=dtype))
raster_coordinates = torch.stack([raster_coordinates[1].reshape(-1), raster_coordinates[0].reshape(-1)])
if not pixel_center_coodinates_are_integer:
    raster_coordinates += .5
raster_coordinates = raster_coordinates.to(device)


sample_coordinates = supersampling_strategy.supersample(raster_coordinates)

### Define default torch renderer

In [ ]:
def render_lines_pt(lines_batch, samples=None, uint=False):
    rasters = render_lines(lines_batch[:, :, 0], lines_batch[:, :, 1], lines_batch[:, :, 2], lines_batch[:, :, 3], lines_batch[:, :, 4], sample_coordinates, samples=samples)
    # sum all lines
    rasters = rasters.sum(1, keepdim=True, dtype=rasters.dtype)
    rasters = supersampling_strategy.subsample(rasters).reshape(-1, padded_h, padded_w)
    if uint:
        return np.uint8(torch.clamp((1 - rasters) * 255, 0, 255).detach().cpu().numpy())
    else:
        return rasters
    
    
def render_lines_skeleton(lines_batch):
    return np.stack(list(map(lambda lines: render_skeleton(lines, [w, h], data_representation='vahe', padding=padding), lines_batch.detach().cpu().numpy())))

## Potentials

### Ground truth and initial vector

In [ ]:
gt_lines_n = 10
vector_lines_n = 10

In [ ]:
gt_lines = np.asarray([get_random_line(h, w) for _ in range(gt_lines_n)])

In [ ]:
gt_lines_n = 2

gt_lines = np.asarray([
    [32, 60, 32, 4, 3],
#    [4, 32, 60, 32, 2]
])

#gt_lines = np.asarray([
#    [10, 30, 90, 30, 3],
#    [10, 35, 90, 35, 2]
#])

In [ ]:
raster_np = render({PT_LINE: gt_lines}, [w, h], data_representation='vahe')
raster = torch.from_numpy(raster_np.astype(np.float32)).to(device)
rasters_batch = (1 - raster/255)[None]
Image.fromarray(raster_np)

In [ ]:
gt_lines_batch = torch.from_numpy(gt_lines.astype(np.float32)[None]).to(device)
Image.fromarray(render_lines_pt(gt_lines_batch, uint=True)[0])

In [ ]:
initial_vector = np.asarray([get_random_line(h, w) for _ in range(vector_lines_n)])
initial_vector = gt_lines + np.random.normal(size=gt_lines.shape) * np.asarray([1, 1, 1, 1, 1/5]) * 10

In [ ]:
initial_vector = gt_lines[:2].copy()
initial_vector[0] = [10, 20, 40, 40, 1]
### initial_vector[1] = [50, 70, 20, 60, 3]
#initial_vector[1] = [70, 50, 80, 50, 3.5]
#initial_vector[1] = [45, 20, 60, 40, 3.5]
#initial_vector[1] = initial_vector[0].copy()
#initial_vector[0][3] -= 20
#initial_vector[1][1] += 20

In [ ]:
## #initial_vector = np.asarray([[0, h/2, w, h/2, 1]])
## x, y = np.random.random(2) * [w, h]
## initial_vector = np.asarray([[x, y, x+1, y+1, 1]])

In [ ]:
lines_n = len(initial_vector)
initial_vector = initial_vector[None]

Image.fromarray(render({PT_LINE: initial_vector[0]}, [w, h], data_representation='vahe'))

### Energies

In [ ]:
# Parameters
R_close = 1
R_far = 32
far_weight = 1 / 50
close_weight = 1 - far_weight

collinearity_beta = 1 / ( (np.abs(np.cos(15 * np.pi / 180)) - 1)**2 )
collinearity_field_weight = 2


def line_to_point_energy_canonical(line_length, line_halfwidth, point_x, point_y, R):
    r"""Gives the total energy of interaction of a line and a point
     for a point-to-point interaction energy given by `-exp(-r**2/R**2)`.
    Coordinates of the point `point_x` and `point_y` correspond to the coordiante system
     where one end of the line is in the origin and the other is on the positive side of y axis.
    """
    return (
        torch.erf((line_length - point_y) / R) + torch.erf(point_y / R)
        ) * (
        torch.erf((line_halfwidth - point_x) / R) + torch.erf((line_halfwidth + point_x) / R)
        ) * (R ** 2)


def line_to_point_energy(lines_batch, point_charges):
    r"""For each line in `lines_batch` gives the total energy of interaction of this line with `point_charges`.
    
    Parameters
    ----------
    lines_batch : torch.Tensor
        of shape [batch_size, lines_n, params_n]
    
    point_charges : torch.Tensor
        of shape [batch_size, lines_n, rasters_n] or [**, rasters_n] broadcastable to this shape
    """
    batch_size, lines_n = lines_batch.shape[:2]
    rasters_n = point_charges.shape[-1]
    
    # Get parameters of the lines
    x1 = lines_batch[..., 0]
    y1 = lines_batch[..., 1]
    x2 = lines_batch[..., 2]
    y2 = lines_batch[..., 3]
    half_width = lines_batch[..., 4] / 2
    lx = x2 - x1
    ly = y2 - y1
    length = torch.sqrt(lx**2 + ly**2)
    nonzero_length = torch.max(length, torch.full([1], division_epsilon, dtype=length.dtype, device=length.device))
    lx = lx / nonzero_length
    ly = ly / nonzero_length
    del lines_batch, x2, y2, nonzero_length
    
    # Broadcast tensors
    x1, y1, half_width, lx, ly, length, point_charges, raster_x, raster_y = torch.broadcast_tensors(x1.unsqueeze(-1), y1.unsqueeze(-1), half_width.unsqueeze(-1), lx.unsqueeze(-1), ly.unsqueeze(-1), length.unsqueeze(-1), point_charges, raster_coordinates[0], raster_coordinates[1])
    
    # Translate points to canonical coordinate systems of the lines
    translated_raster_x = raster_x - x1
    translated_raster_y = raster_y - y1
    del raster_x, raster_y, x1, y1
    canonical_raster_x = translated_raster_x * ly - translated_raster_y * lx
    canonical_raster_y = translated_raster_x * lx + translated_raster_y * ly
    del translated_raster_x, translated_raster_y, ly, lx
    
    # Calculate energies per line
    energies = line_to_point_energy_canonical(length, half_width, canonical_raster_x, canonical_raster_y, R_close) * close_weight \
             + line_to_point_energy_canonical(length, half_width, canonical_raster_x, canonical_raster_y, R_far) * far_weight
    del canonical_raster_x, canonical_raster_y, length, half_width
    energies = torch.sum(energies * point_charges, dim=-1)
    del point_charges
    return energies
    

def line_to_vector_energy(lines_batch, vector_field):
    r"""For each line in `lines_batch` gives the total energy of interaction of this line with `vector_field`.
    
    Parameters
    ----------
    lines_batch : torch.Tensor
        of shape [batch_size, lines_n, params_n]
    
    vector_field : torch.Tensor
        of shape [2, batch_size, lines_n, rasters_n]
    """
    batch_size, lines_n = lines_batch.shape[:2]
    rasters_n = vector_field.shape[-1]
    
    # Get parameters of the lines
    x1 = lines_batch[..., 0]
    y1 = lines_batch[..., 1]
    x2 = lines_batch[..., 2]
    y2 = lines_batch[..., 3]
    half_width = lines_batch[..., 4] / 2
    lx = x2 - x1
    ly = y2 - y1
    length = torch.sqrt(lx**2 + ly**2)
    nonzero_length = torch.max(length, torch.full([1], division_epsilon, dtype=length.dtype, device=length.device))
    lx = lx / nonzero_length
    ly = ly / nonzero_length
    del lines_batch, x2, y2, nonzero_length
    
    # Broadcast tensors
    x1, y1, half_width, lx, ly, length, raster_x, raster_y = torch.broadcast_tensors(x1.unsqueeze(-1), y1.unsqueeze(-1), half_width.unsqueeze(-1), lx.unsqueeze(-1), ly.unsqueeze(-1), length.unsqueeze(-1), raster_coordinates[0], raster_coordinates[1])
    
    # Calculate intensities of vector field interactions and total field interaction intensities
    vector_field_norm = torch.norm(vector_field, dim=0)
    vector_field_norm += division_epsilon
    vector_field /= vector_field_norm
    vector_field_interaction_intensities = vector_field[0] * lx + vector_field[1] * ly
    del vector_field
    vector_field_interaction_intensities = torch.exp(-(vector_field_interaction_intensities.abs() - 1)**2 * collinearity_beta) * vector_field_norm
    del vector_field_norm
    
    # Translate points to canonical coordinate systems of the lines
    translated_raster_x = raster_x - x1
    translated_raster_y = raster_y - y1
    del raster_x, raster_y, x1, y1
    canonical_raster_x = translated_raster_x * ly - translated_raster_y * lx
    canonical_raster_y = translated_raster_x * lx + translated_raster_y * ly
    del translated_raster_x, translated_raster_y, ly, lx
    
    # Calculate energies per line
    energies = line_to_point_energy_canonical(length, half_width, canonical_raster_x, canonical_raster_y, R_close)
    del canonical_raster_x, canonical_raster_y, length, half_width
    energies = torch.sum(energies * vector_field_interaction_intensities, dim=-1)
    del vector_field_interaction_intensities
    return energies * collinearity_field_weight

In [ ]:
def mean_field_energy_lines(lines_batch, rasters_batch, empty_charge=empty_charge, close_range_weight=2*(1/.5), elementary_halfwidth=elementary_halfwidth, visibility_padding=visibility_padding):
    r"""...
    Algorithm is (for each batch):
    1. Render each line on binary supersample grid
    2. Sum (OR) individual renderings -- this is total positive charge field
    3. For each line calculate the sum from step 2 minus the rendering of this line from step 1
       -- this is the excess positive charge field for this line
    4. Subsample the renderings from step 3
    5. Subtract the actual raster from the subsampled renderings from step 4
       -- this is the excess charge field for each line
    
    Steps 6-8 are needed to avoid local minima.
    
    6. For each line calculate coordinates of each excess charge in the coordinate system of this line,
       where the y axis is aligned along the length, the x axis is aligned along the width,
       and the origin is in the center of the line
    7. For each line find the largest rectangle aligned along this line and filled with nonempty pixels only
    7..Such rectangle can be non unique, so define it like this:
    7.1. Select all empty pixels within `elementary_halfwidth` around the direction of the line, i.e |x| <= `elementary_halfwidth`
    7.2. Find the  pixels with minimal positive and maximal negative y coordinate
         -- these pixels correspond to the 'y' edges of the rectangle
    7.3. Select all empty pixels within the 'y' edges of the rectangle
    7.4. Find the pixels with minimal absolute x coordinate
         -- these pixels correspond to the 'x' edges of the rectangle
    8. Weight the excess charge within the rectangle additionally
    
    9. For each line calculate the energy of its interaction with the excess raster field
    
    Parameters
    ----------
    lines_batch : torch.Tensor
        of shape [batch_size, lines_n, params_n]
    
    rasters_batch : torch.Tensor
        of shape [batch_size, rasters_n]
        
    close_range_weight : number
        Should be twice the inverse of the lowest shading value.
    """
    batch_size, lines_n = lines_batch.shape[:2]
    x1 = lines_batch[..., 0]
    y1 = lines_batch[..., 1]
    x2 = lines_batch[..., 2]
    y2 = lines_batch[..., 3]
    half_width = lines_batch[..., 4] / 2
    lx = x2 - x1
    ly = y2 - y1
    length = torch.sqrt(lx**2 + ly**2)
    nonzero_length = torch.max(length, torch.full([1], division_epsilon, dtype=length.dtype, device=length.device))
    lx = lx / nonzero_length
    ly = ly / nonzero_length
    del nonzero_length
    
    with torch.no_grad():
        # 1. Render each line on binary supersample grid
        individual_rasterizations = render_lines(x1, y1, x2, y2, lines_batch[:, :, 4], sample_coordinates)
        
        # 2. Sum (OR) individual renderings -- this is total positive charge field
        patch_rasterizations = individual_rasterizations.sum(1, dtype=individual_rasterizations.dtype)
        
        # 3. For each line calculate the sum from step 2 minus the rendering of this line from step 1
        #    -- this is the excess positive charge field for this line
        others_rasterizations = individual_rasterizations ^ patch_rasterizations.unsqueeze(1)
        del patch_rasterizations, individual_rasterizations
            
        # 4. Subsample the renderings from step 3
        others_rasterizations = supersampling_strategy.subsample(others_rasterizations, dtype=dtype)
        
        # 5. Subtract the actual raster from the subsampled renderings from step 4
        #    -- this is the excess charge field for each line
        excess_raster = others_rasterizations
        rasters_batch = rasters_batch.type(dtype).reshape(batch_size, -1).unsqueeze(1)
        excess_raster -= rasters_batch
        del others_rasterizations
        
        # 6. For each line calculate coordinates of each excess charge in the coordinate system of this line,
        #    where the y axis is aligned along the length, the x axis is aligned along the width,
        #    and the origin is in the center of the line
        cx = (x1 + x2) / 2
        cy = (y1 + y2) / 2
        del x1, x2, y1, y2
        cx, cy, lx, ly, excess_raster, rasters_batch, raster_x, raster_y = torch.broadcast_tensors(cx.unsqueeze(-1), cy.unsqueeze(-1), lx.unsqueeze(-1), ly.unsqueeze(-1), excess_raster, rasters_batch, raster_coordinates[0], raster_coordinates[1])

        # Translate points to canonical coordinate systems of the lines
        translated_raster_x = raster_x - cx
        translated_raster_y = raster_y - cy
        del raster_x, raster_y, cx, cy
        canonical_raster_x_abs = translated_raster_x * ly
        canonical_raster_x_abs -= translated_raster_y * lx
        canonical_raster_x_abs.abs_()
        canonical_raster_y = translated_raster_x * lx
        canonical_raster_y += translated_raster_y * ly
        del translated_raster_x, translated_raster_y, lx, ly
    
        # 7. For each line find the largest rectangle aligned along this line and filled with nonempty pixels only
        #    Such rectangle can be non unique, so define it like this:
        # 7.1. Select all empty pixels within `elementary_halfwidth` around the direction of the line, i.e |x| <= `elementary_halfwidth`
        # 7.2. Find the  pixels with minimal positive and maximal negative y coordinate
        #      -- these pixels correspond to the 'y' edges of the rectangle
        empty = rasters_batch <= empty_charge
        del rasters_batch
        candidates = canonical_raster_x_abs <= elementary_halfwidth
        candidates &= empty
        points_to_the_right = canonical_raster_y >= 0
        candidates_one_side = points_to_the_right & candidates
        assert candidates_one_side.any(dim=-1).all(), 'Couldn\'t find any empty pixel to the right'
        max_y = canonical_raster_y.masked_fill(~candidates_one_side, np.inf)
        max_y = max_y.min(dim=-1)[0]
        candidates_one_side = ~points_to_the_right
        del points_to_the_right
        candidates_one_side &= candidates
        assert candidates_one_side.any(dim=-1).all(), 'Couldn\'t find any empty pixel to the left'
        del candidates
        min_y = canonical_raster_y.masked_fill(~candidates_one_side, -np.inf)
        del candidates_one_side
        min_y = min_y.max(dim=-1)[0]
        
        # 7.3. Select all empty pixels within the 'y' edges of the rectangle
        within_y_edges = canonical_raster_y > min_y.unsqueeze(-1)
        within_y_edges &= canonical_raster_y < max_y.unsqueeze(-1)
        candidates = within_y_edges & empty
        del within_y_edges
        
        # 7.4. Find the pixels with minimal absolute x coordinate
        #      -- these pixels correspond to the 'x' edges of the rectangle
        max_x = canonical_raster_x_abs.masked_fill(~candidates, np.inf)
        max_x = max_x.min(dim=-1)[0]
        max_x.masked_fill_(~torch.isfinite(max_x), 0)
        
        # 8. Weight the excess charge within the rectangle additionally
        #    This is needed to avoid local minima of energy that are not optimal for the size energy
        visible_excess_charge = ~empty
        del empty
        visible_excess_charge &= canonical_raster_y <= (max_y.unsqueeze(-1) + visibility_padding)
        del max_y
        visible_excess_charge &= canonical_raster_y >= (min_y.unsqueeze(-1) - visibility_padding)
        del canonical_raster_y, min_y
        visible_excess_charge &= canonical_raster_x_abs <= (max_x.unsqueeze(-1) + visibility_padding)
        del canonical_raster_x_abs, max_x
        visible_excess_charge &= excess_raster < 0
        excess_raster = excess_raster.where(~visible_excess_charge, excess_raster*close_range_weight)
        del visible_excess_charge
        
    # 9. For each line calculate the energy of its interaction with the excess raster field
    mean_field_energy = line_to_point_energy(lines_batch, excess_raster).sum(-1).mean()
    del excess_raster
    return mean_field_energy

In [ ]:
def mean_vector_field_energy_lines(lines_batch):
    r"""...
    Algorithm is (for each batch):
    1. Render each line on binary supersample grid and subsample
    2. Put unit vector charge, aligned along the line, into every pixel shaded by the line
       -- this is its vector charge field
    3. Sum individual vector charge fields
    4. For each line calculate total complementary vector charge field
    5. For each line calculate the energy of its interaction with complementary vector charge field
     
    Parameters
    ----------
    lines_batch : torch.Tensor
        of shape [batch_size, lines_n, params_n]
    """
    batch_size, lines_n = lines_batch.shape[:2]
    
    with torch.no_grad():
        x1 = lines_batch[..., 0]
        y1 = lines_batch[..., 1]
        x2 = lines_batch[..., 2]
        y2 = lines_batch[..., 3]
        
        # 1. Render each line on binary supersample grid and subsample
        individual_rasterizations = render_lines(x1, y1, x2, y2, lines_batch[:, :, 4], sample_coordinates)
        individual_rasterizations = supersampling_strategy.subsample(individual_rasterizations, dtype=dtype)
        
        # 2. Put unit vector charge, aligned along the line, into every pixel shaded by the line
        #    -- this is its vector charge field
        l_dir = torch.nn.functional.normalize(torch.stack([x2 - x1, y2 - y1]), dim=0, eps=division_epsilon)
        del x1, x2, y1, y2
        individual_vector_fields = l_dir.unsqueeze(-1) * individual_rasterizations
        del individual_rasterizations, l_dir

        # 3. Sum individual vector charge fields
        patch_vector_fields = individual_vector_fields.sum(2)
    
        # 4. For each line calculate total complementary vector charge field
        complimentary_vector_fields = patch_vector_fields.unsqueeze(2) - individual_vector_fields
        del patch_vector_fields, individual_vector_fields
    
    # 5. For each line calculate the energy of its interaction with complementary vector charge field
    mean_field_energy = line_to_vector_energy(lines_batch, complimentary_vector_fields).sum(-1).mean()
    del complimentary_vector_fields
    return mean_field_energy

In [ ]:
def size_energy(lines_batch, rasters_batch, empty_charge=empty_charge, elementary_halfwidth=elementary_halfwidth, visibility_padding=visibility_padding):
    r"""...
    Algorithm is (for each batch):
    1. Render each line on binary supersample grid
    2. Sum (OR) individual renderings -- this is total positive charge field
    3. Subsample the total positive charge fields and individual rasterizations
    4. Subtract the actual raster from the total positive charge field from step 3
       to get the total excess raster for each line
    
    Steps 5-7 are needed to avoid local minima
    
    5. For each line calculate coordinates of each excess charge in the coordinate system of this line,
       where the y axis is aligned along the length, the x axis is aligned along the width,
       and the origin is in the center of the line
    6. For each line find the largest rectangle aligned along this line and filled with nonempty pixels only
       Such rectangle can be non unique, so define it like this:
    6.1. Select all empty pixels within elementary halfwidth around the direction of the line, i.e |x| <= elementary_halfwidth
         elementary halfwidth is e.g 1/2
    6.2. Find the  pixels with minimal positive and maximal negative y coordinate
         -- these pixels correspond to the 'y' edges of the rectangle
    6.3. Select all empty pixels within the 'y' edges of the rectangle
    6.4. Find the pixels with minimal absolute x coordinate
         -- these pixels correspond to the 'x' edges of the rectangle
    7. Limit the visible excess charge of this line to the pixels within its rectangle
       and add all pixels from its individual subsampled rendering
       -- this is the excess charge that the line interacts with
    
    8. For each line calculate the energy of its interaction with the excess raster field
    
    Parameters
    ----------
    lines_batch : torch.Tensor
        of shape [batch_size, lines_n, params_n]
    
    rasters_batch : torch.Tensor
        of shape [batch_size, rasters_n]
    """
    batch_size, lines_n = lines_batch.shape[:2]
    x1 = lines_batch[..., 0]
    y1 = lines_batch[..., 1]
    x2 = lines_batch[..., 2]
    y2 = lines_batch[..., 3]
    half_width = lines_batch[..., 4] / 2
    lx = x2 - x1
    ly = y2 - y1
    length = torch.sqrt(lx**2 + ly**2)
    nonzero_length = torch.max(length, torch.full([1], division_epsilon, dtype=length.dtype, device=length.device))
    lx = lx / nonzero_length
    ly = ly / nonzero_length
    del nonzero_length
    
    with torch.no_grad():
        # 1. Render each line on binary supersample grid
        individual_rasterizations = render_lines(x1, y1, x2, y2, lines_batch[:, :, 4], sample_coordinates)
        
        # 2. Sum (OR) individual renderings -- this is total positive charge field
        patch_rasterizations = individual_rasterizations.sum(1, dtype=individual_rasterizations.dtype)
        
        # 3. Subsample the total positive charge fields and individual rasterizations
        patch_rasterizations = supersampling_strategy.subsample(patch_rasterizations.unsqueeze(1))
        individual_rasterizations = supersampling_strategy.subsample(individual_rasterizations)
        
        # 4. Subtract the actual raster from the total positive charge field from step 3
        #    to get the total excess raster for each line
        excess_raster = patch_rasterizations
        rasters_batch = rasters_batch.type(dtype).reshape(batch_size, -1).unsqueeze(1)
        excess_raster -= rasters_batch
        del patch_rasterizations
    
        # 5. For each line calculate coordinates of each excess charge in the coordinate system of this line,
        #    where the y axis is aligned along the length, the x axis is aligned along the width,
        #    and the origin is in the center of the line
        cx = (x1 + x2) / 2
        cy = (y1 + y2) / 2
        del x1, x2, y1, y2
        cx, cy, lx, ly, excess_raster, rasters_batch, raster_x, raster_y = torch.broadcast_tensors(cx.unsqueeze(-1), cy.unsqueeze(-1), lx.unsqueeze(-1), ly.unsqueeze(-1), excess_raster, rasters_batch, raster_coordinates[0], raster_coordinates[1])

        # Translate points to canonical coordinate systems of the lines
        translated_raster_x = raster_x - cx
        translated_raster_y = raster_y - cy
        del raster_x, raster_y, cx, cy
        canonical_raster_x_abs = translated_raster_x * ly
        canonical_raster_x_abs -= translated_raster_y * lx
        canonical_raster_x_abs.abs_()
        canonical_raster_y = translated_raster_x * lx
        canonical_raster_y += translated_raster_y * ly
        del translated_raster_x, translated_raster_y, lx, ly
    
        # 6. For each line find the largest rectangle aligned along this line and filled with nonempty pixels only
        #    Such rectangle can be non unique, so define it like this:
        # 6.1. Select all empty pixels within elementary halfwidth around the direction of the line, i.e |x| <= elementary_halfwidth
        #      elementary halfwidth is e.g 1/2
        # 6.2. Find the  pixels with minimal positive and maximal negative y coordinate
        #      -- these pixels correspond to the 'y' edges of the rectangle
        empty = rasters_batch <= empty_charge
        del rasters_batch
        candidates = canonical_raster_x_abs <= elementary_halfwidth
        candidates &= empty
        points_to_the_right = canonical_raster_y >= 0
        candidates_one_side = points_to_the_right & candidates
        assert candidates_one_side.any(dim=-1).all(), 'Couldn\'t find any empty pixel to the right'
        max_y = canonical_raster_y.masked_fill(~candidates_one_side, np.inf)
        max_y = max_y.min(dim=-1)[0]
        candidates_one_side = ~points_to_the_right
        del points_to_the_right
        candidates_one_side &= candidates
        assert candidates_one_side.any(dim=-1).all(), 'Couldn\'t find any empty pixel to the left'
        del candidates
        min_y = canonical_raster_y.masked_fill(~candidates_one_side, -np.inf)
        del candidates_one_side
        min_y = min_y.max(dim=-1)[0]
        
        # 6.3. Select all empty pixels within the 'y' edges of the rectangle
        within_y_edges = canonical_raster_y > min_y.unsqueeze(-1)
        within_y_edges &= canonical_raster_y < max_y.unsqueeze(-1)
        candidates = within_y_edges & empty
        del within_y_edges
        
        # 6.4. Find the pixels with minimal absolute x coordinate
        #      -- these pixels correspond to the 'x' edges of the rectangle
        max_x = canonical_raster_x_abs.masked_fill(~candidates, np.inf)
        max_x = max_x.min(dim=-1)[0]
        max_x.masked_fill_(~torch.isfinite(max_x), 0)
        
        # 7. Limit the visible excess charge of this line to the pixels within its rectangle
        #    and add all pixels from its individual subsampled rendering
        #    -- this is the excess charge that the line interacts with
        visible_excess_charge = ~empty
        del empty
        visible_excess_charge &= canonical_raster_y <= (max_y.unsqueeze(-1) + visibility_padding)
        del max_y
        visible_excess_charge &= canonical_raster_y >= (min_y.unsqueeze(-1) - visibility_padding)
        del canonical_raster_y, min_y
        visible_excess_charge &= canonical_raster_x_abs <= (max_x.unsqueeze(-1) + visibility_padding)
        del canonical_raster_x_abs, max_x
        excess_raster = excess_raster.where(visible_excess_charge, individual_rasterizations)
        del visible_excess_charge, individual_rasterizations
        ### ax_debug.imshow(excess_raster[0, 0].detach().cpu().reshape(padded_h, padded_w))
        
    # 8. For each line calculate the energy of its interaction with the excess raster field
    mean_field_energy = line_to_point_energy(lines_batch, excess_raster).sum(-1).mean()
    del excess_raster
    return mean_field_energy

### Reinitialization of bad predictions, snapping, and redundancy filtering

In [ ]:
def reinit_excess_lines(cx, cy, width, length, excess_raster, patches_to_consider, min_raster_to_fill=.5, min_width=1/8, initial_width=1, initial_length=1):
    r"""
    Algorithm is:
    1. In each patch find the maximum value of excess raster
       that is not already covered by some vector primitive
    2. Find the patches for which this value is larger than `min_raster_to_fill`
       and that are from `patches_to_consider`
       -- only these patches need reinitialization
    3. In every such patch find the line with the minimal width
    4. Find the patches for which this value is lower than `min_width`
         (the patch has invisible lines that are not already 'working')
       and that need reinitialization
       -- only these patches will be reinitialized
    5. In every such patch put the line with the minimal width from step 3
       into the position maximum excess raster from step 1
       and reinitialize the length and width of this line
    """
    with torch.no_grad():
        # 1. In each patch find the maximum value of excess raster
        #    that is not already covered by some vector primitive
        max_excess, max_excess_id = torch.max(excess_raster, dim=-1)
        # 2. Find the patches for which this value is larger than `min_raster_to_fill`
        #    and that are from `patches_to_consider`
        #    -- only these patches need reinitialization
        patches_to_reinit = torch.tensor(patches_to_consider, dtype=torch.bool, device=cx.device)
        patches_to_consider_to_reinit = max_excess >= min_raster_to_fill
        patches_to_reinit.masked_scatter_(patches_to_reinit, patches_to_consider_to_reinit)
        if patches_to_reinit.sum() == 0:
            return
        # 3. In every such patch find the line with the minimal width
        min_lines_width, lines_to_reinit = torch.min(width[patches_to_reinit], dim=-1)
        # 4. Find the patches for which this value is lower than `min_width`
        #      (the patch has invisible lines that are not already 'working')
        #    and that need reinitialization
        #    -- only these patches will be reinitialized
        patches_to_reinit_with_excess_lines = min_lines_width < min_width
        patches_to_consider_to_reinit[patches_to_consider_to_reinit] &= patches_to_reinit_with_excess_lines
        patches_to_reinit[patches_to_reinit] &= patches_to_reinit_with_excess_lines
        if patches_to_reinit.sum() == 0:
            return
        lines_to_reinit = lines_to_reinit[patches_to_reinit_with_excess_lines]
        # 5. In every such patch put the line with the minimal width from step 3
        #    into the position maximum excess raster from step 1
        #    and reinitialize the length and width of this line 
        cx.data[patches_to_reinit, lines_to_reinit] = raster_coordinates[0, max_excess_id[patches_to_consider_to_reinit]]
        cy.data[patches_to_reinit, lines_to_reinit] = raster_coordinates[1, max_excess_id[patches_to_consider_to_reinit]]
        length.data[patches_to_reinit, lines_to_reinit] = initial_length
        width.data[patches_to_reinit, lines_to_reinit] = initial_width

In [ ]:
def snap_lines(cx, cy, theta, length, width, pos_optimizer, size_optimizer, width_threshold=1/4, coord_threshold=1.5, direction_threshold=5, min_linear=min_linear):
    r"""
    Algorith is for each 'this' line:
    1.C. Select the collinear other lines
    1.C.21. Among them select the lines with p2 == this p1
    1.C.21.W. Among them select the lines with same widths
              With the other parts of the algorithm, chances are low that such line is nonunique for 'this' line
    2.C.21. Snap the other line with 'this' line, collapse 'this' line,
            and keep track of the modified and collapsed lines
     Do the same for other variants of 1-2
    3. Prevent rocking after snaps      
    4. Prevent size freezing after snaps
    5. Reset collapsed lines
    """
    modified_lines = torch.full(cx.shape, False, dtype=torch.bool, device=cx.device)
    collapsed_lines = torch.full(cx.shape, False, dtype=torch.bool, device=cx.device)
    with torch.no_grad():
        # For each 'this' line
        for line_i in range(cx.shape[1] - 1):
            length_others = length[:, line_i+1:]
            width_others = width[:, line_i+1:]
            theta_others = theta[:, line_i+1:]
            cos_others = torch.cos(theta_others)
            sin_others = torch.sin(theta_others)
            cx_others = cx[:, line_i+1:]
            cy_others = cy[:, line_i+1:]
            x1_others = cx_others - length_others * cos_others / 2
            y1_others = cy_others - length_others * sin_others / 2
            x2_others = cx_others + length_others * cos_others / 2
            y2_others = cy_others + length_others * sin_others / 2

            length_this = length[:, line_i:line_i+1].expand_as(length_others)
            width_this = width[:, line_i:line_i+1].expand_as(width_others)
            theta_this = theta[:, line_i:line_i+1].expand_as(theta_others)
            cos_this = torch.cos(theta_this)
            sin_this = torch.sin(theta_this)
            cx_this = cx[:, line_i:line_i+1].expand_as(cx_others)
            cy_this = cy[:, line_i:line_i+1].expand_as(cy_others)
            x1_this = cx_this - length_this * cos_this / 2
            y1_this = cy_this - length_this * sin_this / 2
            x2_this = cx_this + length_this * cos_this / 2
            y2_this = cy_this + length_this * sin_this / 2
            
            cos_theta_dif = torch.cos(theta_others - theta_this)
            cos_threshold = np.cos(direction_threshold * np.pi / 180)
            
            not_modified_this = torch.full(length_others.shape, True, dtype=torch.bool, device=length_others.device)
            
            # 1.C. Select the collinear other lines
            close_directions = cos_theta_dif > cos_threshold
            
            # 1.C.21. Among them select the lines with p2 == this p1
            close_other2_this1 = close_directions.clone()
            close_other2_this1[close_directions] &= torch.abs(x2_others[close_directions] - x1_this[close_directions]) <= coord_threshold
            close_other2_this1[close_directions] &= torch.abs(y2_others[close_directions] - y1_this[close_directions]) <= coord_threshold
            
            # 1.C.21.W. Among them select the lines with same widths
            #  With the other parts of the algorithm, chances are low that such line is nonunique for 'this' line
            close_width = close_other2_this1.clone()
            close_width[close_other2_this1] &= torch.abs(width_others[close_other2_this1] - width_this[close_other2_this1]) < width_threshold

            # 2.C.21. Snap the other line with 'this' line,
            new_x2_others = x2_this[close_width]
            new_y2_others = y2_this[close_width]
            cx_others.data[close_width] = (new_x2_others + x1_others[close_width]) / 2
            cy_others.data[close_width] = (new_y2_others + y1_others[close_width]) / 2
            length_others.data[close_width] = torch.sqrt((new_x2_others - x1_others[close_width])**2 + (new_y2_others - y1_others[close_width])**2) 
            #         collapse 'this' line
            length_this.data[close_width] = min_linear
            width_this.data[close_width] = min_linear
            #         and keep track of the modified and collapsed lines
            modified_lines[:, line_i+1:][close_width] = True
            collapsed_lines[:, line_i][close_width.max(dim=-1)[0]] = True
            not_modified_this[close_width.max(dim=-1)[0]] = False
            close_directions[close_width.max(dim=-1)[0]] = False
            del new_x2_others, new_y2_others, close_other2_this1, close_width

            # 1.C.12. other's p1 = this p2
            close_other1_this2 = close_directions.clone()
            close_other1_this2[close_directions] &= torch.abs(x1_others[close_directions] - x2_this[close_directions]) <= coord_threshold
            close_other1_this2[close_directions] &= torch.abs(y1_others[close_directions] - y2_this[close_directions]) <= coord_threshold
                        
            # 1.C.12.W. same widths
            close_width = close_other1_this2.clone()
            close_width[close_other1_this2] &= torch.abs(width_others[close_other1_this2] - width_this[close_other1_this2]) < width_threshold
            
            # 2.C.12. Snap the other line with 'this' line,
            new_x1_others = x1_this[close_width]
            new_y1_others = y1_this[close_width]
            cx_others.data[close_width] = (new_x1_others + x2_others[close_width]) / 2
            cy_others.data[close_width] = (new_y1_others + y2_others[close_width]) / 2
            #         collapse 'this' line
            length_others.data[close_width] = torch.sqrt((x2_others[close_width] - new_x1_others)**2 + (y2_others[close_width] - new_y1_others)**2)
            length_this.data[close_width] = min_linear
            width_this.data[close_width] = min_linear
            #         and keep track of the modified and collapsed lines
            modified_lines[:, line_i+1:][close_width] = True
            collapsed_lines[:, line_i][close_width.max(dim=-1)[0]] = True
            not_modified_this[close_width.max(dim=-1)[0]] = False
            del new_x1_others, new_y1_others, close_other1_this2, close_width
            
            # 1.A. anticollinear
            close_directions = (cos_theta_dif < -cos_threshold) & not_modified_this
            
            # 1.A.22. other's p2 = this p2
            close_other2_this2 = close_directions.clone()
            close_other2_this2[close_directions] &= torch.abs(x2_others[close_directions] - x2_this[close_directions]) <= coord_threshold
            close_other2_this2[close_directions] &= torch.abs(y2_others[close_directions] - y2_this[close_directions]) <= coord_threshold
            
            # 1.A.22.W. same widths
            close_width = close_other2_this2.clone()
            close_width[close_other2_this2] &= torch.abs(width_others[close_other2_this2] - width_this[close_other2_this2]) < width_threshold

            # 2.A.22. Snap the other line with 'this' line,
            new_x2_others = x1_this[close_width]
            new_y2_others = y1_this[close_width]
            cx_others.data[close_width] = (new_x2_others + x1_others[close_width]) / 2
            cy_others.data[close_width] = (new_y2_others + y1_others[close_width]) / 2
            length_others.data[close_width] = torch.sqrt((new_x2_others - x1_others[close_width])**2 + (new_y2_others - y1_others[close_width])**2) 
            #         collapse 'this' line
            length_this.data[close_width] = min_linear
            width_this.data[close_width] = min_linear
            #         and keep track of the modified and collapsed lines
            modified_lines[:, line_i+1:][close_width] = True
            collapsed_lines[:, line_i][close_width.max(dim=-1)[0]] = True
            not_modified_this[close_width.max(dim=-1)[0]] = False
            close_directions[close_width.max(dim=-1)[0]] = False
            del new_x2_others, new_y2_others, close_other2_this2, close_width

            # 1.A.11. other's p1 = this p1
            close_other1_this1 = close_directions.clone()
            close_other1_this1[close_directions] &= torch.abs(x1_others[close_directions] - x1_this[close_directions]) <= coord_threshold
            close_other1_this1[close_directions] &= torch.abs(y1_others[close_directions] - y1_this[close_directions]) <= coord_threshold
                        
            # 1.A.11.W. same widths
            close_width = close_other1_this1.clone()
            close_width[close_other1_this1] &= torch.abs(width_others[close_other1_this1] - width_this[close_other1_this1]) < width_threshold
            
            # 2.A.11. Snap the other line with 'this' line,
            new_x1_others = x2_this[close_width]
            new_y1_others = y2_this[close_width]
            cx_others.data[close_width] = (new_x1_others + x2_others[close_width]) / 2
            cy_others.data[close_width] = (new_y1_others + y2_others[close_width]) / 2
            length_others.data[close_width] = torch.sqrt((x2_others[close_width] - new_x1_others)**2 + (y2_others[close_width] - new_y1_others)**2)
            #         collapse 'this' line
            length_this.data[close_width] = min_linear
            width_this.data[close_width] = min_linear
            #         and keep track of the modified and collapsed lines
            modified_lines[:, line_i+1:][close_width] = True
            collapsed_lines[:, line_i][close_width.max(dim=-1)[0]] = True
            del new_x1_others, new_y1_others, close_other1_this1, close_width, close_directions
                  
    # 3. Prevent rocking after snaps      
    angle_damper = pos_optimizer.state[theta]['exp_avg_sq'].new_full([1], 1e6)
    pos_optimizer.state[theta]['exp_avg'].data[modified_lines] = 0
    pos_optimizer.state[theta]['exp_avg_sq'].data[modified_lines] = pos_optimizer.state[theta]['exp_avg_sq'].data[modified_lines].max(angle_damper)
    pos_optimizer.state[cx]['exp_avg'].data[modified_lines] = 0
    pos_optimizer.state[cx]['exp_avg_sq'].data[modified_lines] = 0
    pos_optimizer.state[cy]['exp_avg'].data[modified_lines] = 0
    pos_optimizer.state[cy]['exp_avg_sq'].data[modified_lines] = 0
    # 4. Prevent size freezing after snaps
    size_optimizer.state[length]['exp_avg'].data[modified_lines] = 0
    size_optimizer.state[length]['exp_avg_sq'].data[modified_lines] = 0
    size_optimizer.state[width]['exp_avg'].data[modified_lines] = 0
    size_optimizer.state[width]['exp_avg_sq'].data[modified_lines] = 0
    # 5. Reset collapsed lines
    pos_optimizer.state[cx]['exp_avg'].data[collapsed_lines] = 0
    pos_optimizer.state[cx]['exp_avg_sq'].data[collapsed_lines] = 0
    pos_optimizer.state[cy]['exp_avg'].data[collapsed_lines] = 0
    pos_optimizer.state[cy]['exp_avg_sq'].data[collapsed_lines] = 0
    pos_optimizer.state[theta]['exp_avg'].data[collapsed_lines] = 0
    pos_optimizer.state[theta]['exp_avg_sq'].data[collapsed_lines] = 0
    size_optimizer.state[length]['exp_avg'].data[collapsed_lines] = 0
    size_optimizer.state[length]['exp_avg_sq'].data[collapsed_lines] = 0
    size_optimizer.state[width]['exp_avg'].data[collapsed_lines] = 0
    size_optimizer.state[width]['exp_avg_sq'].data[collapsed_lines] = 0

In [ ]:
def collapse_redundant_lines(cx, cy, theta, length, width, patches_to_consider, enum_type=torch.int8, min_linear=min_linear):
    r"""
    Algorithm is:
    1. Render each line on binary supersample grid
    2. Sum (OR) individual renderings
    3. For each line check if the line is redundant:
       Remember, that our renderings on the subsample grid are binary.
       For each pixel where the current line is rendered we subtract 2 from the total number of lines in this pixel.
       If this line is the only one in this pixel, then the result is negative;
        if this line is not the only one, then the result is nonnegative.
       The line is redundant iff there are no pixels where it is not the only one
    4. Remove redundant lines from the total rendering
    """
    # assert lines_n < 128, 'Risk of overflow! Change int8 to something else'
    lines_n = cx.shape[1]
    with torch.no_grad():
        x2 = cx.data[patches_to_consider] + length.data[patches_to_consider] * torch.cos(theta.data[patches_to_consider]) / 2
        x1 = cx.data[patches_to_consider] - length.data[patches_to_consider] * torch.cos(theta.data[patches_to_consider]) / 2
        y2 = cy.data[patches_to_consider] + length.data[patches_to_consider] * torch.sin(theta.data[patches_to_consider]) / 2
        y1 = cy.data[patches_to_consider] - length.data[patches_to_consider] * torch.sin(theta.data[patches_to_consider]) / 2
        
        # 1. Render each line on binary supersample grid
        individual_rasterizations = render_lines(x1, y1, x2, y2, width[patches_to_consider], sample_coordinates).type(enum_type)
        
        # 2. Sum (OR) individual renderings
        patch_rasterizations = individual_rasterizations.sum(1, dtype=enum_type)
        
        for line_i in range(lines_n):
            patches_to_work_with = torch.tensor(patches_to_consider, dtype=torch.bool, device=cx.device)
            # 3. Check if a line is redundant.
            # Remember, that our renderings on the subsample grid are binary.
            # For each pixel where the current line is rendered
            # we subtract 2 from the total number of lines in this pixel.
            # If this line is the only one in this pixel, then the result is negative;
            # if this line is not the only one, then the result is nonnegative.
            patch_rasterizations_with_negative_line = individual_rasterizations[:, line_i].clone()
            patch_rasterizations_with_negative_line *= -2
            patch_rasterizations_with_negative_line += patch_rasterizations
            
            # The line is redundant iff there are no pixels where it is not the only one
            line_is_redundant = (patch_rasterizations_with_negative_line >= 0).all(dim=-1)
            del patch_rasterizations_with_negative_line
            
            # 4. Remove redundant lines from the total rendering
            patch_rasterizations[line_is_redundant] -= individual_rasterizations[line_is_redundant, line_i]
            patches_to_work_with[patches_to_work_with] = line_is_redundant
            width[patches_to_work_with, line_i] = min_linear
            length[patches_to_work_with, line_i] = min_linear
        del individual_rasterizations, patch_rasterizations, line_is_redundant

### Parameters constraining

In [ ]:
assert padding - 2 > 0

def constrain_parameters(cx, cy, theta, length, width, canvas_width, canvas_height, size_optimizer, padding=padding-2, min_linear=min_linear, dwarfness_ratio=1):
    r"""
    1. Constrain width and length to be non less than `min_linear` which is nonzero
       to prevent 'dying' of the lines (any position of a zero-sized line is optimal)
    3. Swap width and length for short and wide 'dwarf' lines
    4. Limit positions of the lines to the canvas
       Nonzero padding is used to prevent nonstability for the lines trying to fit super-narrow raster
       (i.e, with small shading value) at the very edge of the canvas
       We use the `size_energy` padding value minus 2
       since `size_energy` needs the line to be at least 2 pixels away from the edge
    5. Limit the length of the line w.r.t the edges of the canvas
    """
    ## # 0. Reset exponential averages for collapsed lines
    ## size_optimizer.state[length]['exp_avg'].data[length <= min_linear] = 0
    ## size_optimizer.state[length]['exp_avg_sq'].data[length <= min_linear] = 0
    ## size_optimizer.state[width]['exp_avg'].data[width <= min_linear] = 0
    ## size_optimizer.state[width]['exp_avg_sq'].data[width <= min_linear] = 0
    
    # 1. Constrain width and length to be non less than `min_linear` which is nonzero
    #    to prevent 'dying' of the lines (any position of a zero-sized line is optimal)
    width.data.clamp_(min=min_linear)
    length.data.clamp_(min=min_linear)
    # 2. Keep theta in [0, 2pi)
    theta.data.remainder_(np.pi * 2)
    
    # 3. Swap width and length for short and wide 'dwarf' lines
    dwarf_lines = width.data > length.data * dwarfness_ratio
    width.data[dwarf_lines], length.data[dwarf_lines] = length.data[dwarf_lines], width.data[dwarf_lines]
    theta.data[dwarf_lines] += np.pi / 2
    size_optimizer.state[length]['exp_avg'].data[dwarf_lines], size_optimizer.state[width]['exp_avg'].data[dwarf_lines] = size_optimizer.state[width]['exp_avg'].data[dwarf_lines], size_optimizer.state[length]['exp_avg'].data[dwarf_lines]
    size_optimizer.state[length]['exp_avg_sq'].data[dwarf_lines], size_optimizer.state[width]['exp_avg_sq'].data[dwarf_lines] = size_optimizer.state[width]['exp_avg_sq'].data[dwarf_lines], size_optimizer.state[length]['exp_avg_sq'].data[dwarf_lines]
    
    # 4. Limit positions of the lines to the canvas
    #    Nonzero padding is used to prevent nonstability for the lines trying to fit super-narrow raster
    #    (i.e, with small shading value) at the very edge of the canvas
    #    We use the `size_energy` padding value minus 2
    #    since `size_energy` needs the line to be at least 2 pixels away from the edge
    cx.data.clamp_(min=-padding, max=canvas_width + padding)
    cy.data.clamp_(min=-padding, max=canvas_height + padding)
    
    # 5. Limit the length of the line w.r.t the edges of the canvas
    epsiloned_2cos = torch.abs(torch.cos(theta.data)) / 2 + division_epsilon
    epsiloned_2sin = torch.abs(torch.sin(theta.data)) / 2 + division_epsilon
    maxl = torch.min(torch.stack([
        (cx.data + padding) / epsiloned_2cos, (canvas_width + padding - cx.data) / epsiloned_2cos,
        (cy.data + padding) / epsiloned_2sin, (canvas_height + padding - cy.data) / epsiloned_2sin
    ], -1), dim=-1)[0]
    length.data = torch.min(length.data, maxl).clamp(min=min_linear)

## Optimization tests

#### Test on real predictions

In [ ]:
test_real = False
if test_real:
    take_batches = slice(70, 90)
    #take_batches = slice(119, 120)
    #take_batches = slice(140, 160)

    first_batch_to_take = 550
    take_batches_n = 400

    rasters_batch = torch.load('/home/ovoinov/Downloads/oleg_images').detach()
    take_batches = []
    while True:
        if rasters_batch[first_batch_to_take].max() > 0:
            take_batches.append(first_batch_to_take)
        if len(take_batches) >= take_batches_n:
            break
        first_batch_to_take += 1
    print(take_batches[-1])
    take_batches = np.asarray(take_batches)
    #take_batches = [take_batches[319]]
    #take_batches = take_batches[[84, 54, 157, 29, 240, 58, 185, 49, 266, 114, 349, 264, 286, 384, 129, 263, 281, 283, 319, 365]]
    #take_batches = take_batches[[163, 286, 350, 281, 263, 287, 162, 384, 349, 285, 284, 85, 319, 324, 185, 366, 365, 383, 259, 49]]
    #take_batches = take_batches[[-3]]
    take_batches = take_batches[[286, 263, 281, 349, 85, 350, 383, 287, 285, 284, 319, 259, 323, 204, 283, 86, 324, 229, 282, 264]]
    #take_batches = take_batches[[6]]
    
    rasters_batch = rasters_batch[take_batches, 0].type(dtype).to(device)
    raster_np = (1 - rasters_batch[0].cpu().numpy()) * 255

    initial_vector = torch.load('/home/ovoinov/Downloads/oleg_lines').detach()[take_batches].cpu()
    removed_lines = initial_vector[..., -1] < .5 * h
    rand_x1 = torch.rand_like(initial_vector[removed_lines, [0]]) * w
    rand_y1 = torch.rand_like(initial_vector[removed_lines, [1]]) * h
    initial_vector[removed_lines, [0]] = rand_x1
    initial_vector[removed_lines, [2]] = rand_x1 + 1
    initial_vector[removed_lines, [1]] = rand_y1
    initial_vector[removed_lines, [3]] = rand_y1 + 1
    initial_vector[removed_lines, [4]] = 2**-8
    initial_vector = initial_vector[..., :5].numpy()

    Image.fromarray(np.hstack([
        np.uint8(np.vstack((1 - rasters_batch.detach().cpu().numpy())*255)),
        np.vstack([render({PT_LINE: lines}, [w, h], data_representation='vahe') for lines in initial_vector])
    ]))
else:
    rasters_batch = np.asarray(Image.open('/home/ovoinov/Downloads/2019-10-17 21.12.33.jpg')).astype(np.float32)[..., 0]
    rasters_batch = (1 - rasters_batch/255)
    rasters_batch = torch.from_numpy(rasters_batch[None]).type(dtype).to(device)
    initial_vector = np.random.rand(1, 20, 5) * [w, h, w, h, 1]

In [ ]:
rasters_batch = torch.nn.functional.pad(rasters_batch, [padding, padding, padding, padding])

#### Initialize vector lines

In [ ]:
lines_batch = torch.from_numpy(initial_vector).type(dtype).to(device)
lines_n = lines_batch.shape[1]

# get the canonical parameters
x1 = lines_batch[:, :, 0]
y1 = lines_batch[:, :, 1]
x2 = lines_batch[:, :, 2]
y2 = lines_batch[:, :, 3]
width = lines_batch[:, :, 4]
X = x2 - x1
Y = y2 - y1
length = torch.sqrt(X**2 + Y**2)
theta = torch.atan2(Y, X)
cx = (x1 + x2) / 2
cy = (y1 + y2) / 2
del x1, x2, y1, y2, X, Y

for canonical_parameter in cx, cy, theta, length, width:
    canonical_parameter.requires_grad = True

pos_optimizer = NonanAdam([cx, cy, theta], lr=1e-1)
size_optimizer = NonanAdam([length, width], lr=1e-1)
# initialize optimizers
(cx + cy + theta + length + width).reshape(-1)[0].backward()
pos_optimizer.zero_grad()
pos_optimizer.step()
size_optimizer.zero_grad()
size_optimizer.step()

patches_to_optimize = np.full(lines_batch.shape[0], True, np.bool) #my_iou_score(vector_rendering, rasters_batch) < .98
cx_final = torch.empty_like(cx)
cy_final = torch.empty_like(cy)
theta_final = torch.empty_like(theta)
length_final = torch.empty_like(length)
width_final = torch.empty_like(width)
lines_batch_final = torch.empty_like(lines_batch)

#### Initialize logging

In [ ]:
iters_n = 10000
plotting = True
log_plotting = False

energies = np.full(iters_n, np.inf)
if plotting:
    ims = []
    fig, [initial_pred_ax, initial_skeleton_ax, refined_ax, refined_skeleton_ax, gt_ax, dif_ax] = plt.subplots(1, 6, figsize=[4 * 6, 4 * len(lines_batch)])

if log_plotting:
    fig = plt.figure(figsize=[4, 4])
    log_data = []
    log_data_x = []
    log_plot_ax = fig.gca()
    log_plot, = log_plot_ax.plot(log_data_x, log_data)

#### Metrics

In [ ]:
import sys

sys.path.append('/home/ovoinov/work/3ddl/vectorization/FloorplanVectorization')
from vectran.metrics.raster_metrics import iou_score


def my_iou_score(vector_rendering, rasters_batch, average=None):
    _vector = ((1 - vector_rendering.detach().cpu()).clamp(0, 1) * 255).type(torch.uint8).numpy()
    _raster = ((1 - rasters_batch.detach().cpu()).clamp(0, 1) * 255).type(torch.uint8).numpy()
    return iou_score(_raster, _vector, binarization='median', average=average)

#### Optimize

In [ ]:
from tqdm import tqdm

In [ ]:
"""
Algorithm is:
1. Reinitialize excess predictions
2. Optimize mean field energy w.r.t position parameters with fixed size parameters
   Apply constraints to the parameters of the lines after this and other parts of iteration
3. Optimize mean field energy w.r.t size parameters
   with fixed positions of the left points of the lines and their orientations
   Penalize collinearity of overlapping lines
4. Optimize mean field energy w.r.t size parameters
   with fixed positions of the right points of the lines and their orientations
   Penalize collinearity of overlapping lines
5. Snap lines that have coinciding ends, close orientations and close widths
6. Prepare the results for logging
7. Find the patches with high enough IOU value and do not optimize them furher
"""

vector_rendering = render_lines_pt(lines_batch.detach())
if plotting:
    im = rasters_batch - vector_rendering
    im_initial_pred = vector_rendering.cpu().numpy()
    im_refined = im_initial_pred
    im_gt = rasters_batch.cpu().numpy()
    im_dif = im.cpu().numpy()
    im_initial_skeleton = render_lines_skeleton(lines_batch.detach())
    im_refined_skeleton = im_initial_skeleton
    
    initial_pred_plot = initial_pred_ax.imshow(np.vstack(im_initial_pred), vmin=0, vmax=1, cmap='gray_r')
    refined_plot = refined_ax.imshow(np.vstack(im_refined), vmin=0, vmax=1, cmap='gray_r')
    gt_plot = gt_ax.imshow(np.vstack(im_gt), vmin=0, vmax=1, cmap='gray_r')
    dif_plot = dif_ax.imshow(np.vstack(im_dif), vmin=-1, vmax=1, cmap='gray_r')
    initial_skeleton_plot = initial_skeleton_ax.imshow(np.vstack(im_initial_skeleton))
    refined_skeleton_plot = refined_skeleton_ax.imshow(np.vstack(im_refined_skeleton))
    
    ax_debug = initial_skeleton_ax


its_time_to_stop = [False]
def plotting_sigint(*args):
    its_time_to_stop[0] = True

for i in tqdm(range(iters_n)):
    try:
        # 1. Reinitialize excess predictions
        #    The meaning of `patches_to_optimize` is explained below, in step 7
        if i % 20 == 0:
            x2 = cx.data + length.data * torch.cos(theta.data) / 2
            x1 = cx.data - length.data * torch.cos(theta.data) / 2
            y2 = cy.data + length.data * torch.sin(theta.data) / 2
            y1 = cy.data - length.data * torch.sin(theta.data) / 2
            lines_batch = torch.stack([x1, y1, x2, y2, width.data], -1)
            lines_batch.data[..., -1][lines_batch[..., -1] < 1/4] = 0
            vector_rendering[patches_to_optimize] = render_lines_pt(lines_batch[patches_to_optimize].detach())
            im = rasters_batch[patches_to_optimize].clone()
            # the line below is explained in `reinit_excess_lines`
            im.masked_fill_(vector_rendering[patches_to_optimize] > 0, 0)
            reinit_excess_lines(cx, cy, width, length, im.reshape(im.shape[0], -1), patches_to_consider=patches_to_optimize)

        # 2. Optimize mean field energy w.r.t position parameters with fixed size parameters
        x2 = cx + length.data * torch.cos(theta) / 2
        x1 = cx - length.data * torch.cos(theta) / 2
        y2 = cy + length.data * torch.sin(theta) / 2
        y1 = cy - length.data * torch.sin(theta) / 2
        lines_batch = torch.stack([x1, y1, x2, y2, width.data], -1)
        mean_field_energy = mean_field_energy_lines(lines_batch[patches_to_optimize], rasters_batch[patches_to_optimize])
        
        pos_optimizer.zero_grad()
        mean_field_energy.backward()
        pos_optimizer.step()
        #    Apply constraints to the parameters of the lines after this and other parts of iteration
        constrain_parameters(cx, cy, theta, length, width, canvas_width=w, canvas_height=h, size_optimizer=size_optimizer)
        
        # 3. Optimize mean field energy w.r.t size parameters
        #    with fixed positions of the left points of the lines and their orientations
        x1 = cx.data - length.data * torch.cos(theta.data) / 2
        y1 = cy.data - length.data * torch.sin(theta.data) / 2
        x2 = x1 + length * torch.cos(theta.data)
        y2 = y1 + length * torch.sin(theta.data)
        lines_batch = torch.stack([x1, y1, x2, y2, width], -1)

        excess_energy = size_energy(lines_batch[patches_to_optimize], rasters_batch[patches_to_optimize])
        #    Penalize collinearity of overlapping lines
        collinearity_energy = mean_vector_field_energy_lines(lines_batch[patches_to_optimize])
        size_optimizer.zero_grad()
        (excess_energy + collinearity_energy).backward()
        size_optimizer.step()
        
        cx.data[patches_to_optimize] = x1.data[patches_to_optimize] + length.data[patches_to_optimize] * torch.cos(theta.data[patches_to_optimize]) / 2
        cy.data[patches_to_optimize] = y1.data[patches_to_optimize] + length.data[patches_to_optimize] * torch.sin(theta.data[patches_to_optimize]) / 2
        constrain_parameters(cx, cy, theta, length, width, canvas_width=w, canvas_height=h, size_optimizer=size_optimizer)

        # 4. Optimize mean field energy w.r.t size parameters
        #    with fixed positions of the right points of the lines and their orientations
        x2 = cx.data + length.data * torch.cos(theta.data) / 2
        y2 = cy.data + length.data * torch.sin(theta.data) / 2
        x1 = x2 - length * torch.cos(theta.data)
        y1 = y2 - length * torch.sin(theta.data)
        lines_batch = torch.stack([x1, y1, x2, y2, width], -1)

        excess_energy = size_energy(lines_batch[patches_to_optimize], rasters_batch[patches_to_optimize])
        #    Penalize collinearity of overlapping lines
        collinearity_energy = mean_vector_field_energy_lines(lines_batch[patches_to_optimize])
        size_optimizer.zero_grad()
        (excess_energy + collinearity_energy).backward()
        size_optimizer.step()
         
        cx.data[patches_to_optimize] = x2.data[patches_to_optimize] - length.data[patches_to_optimize] * torch.cos(theta.data[patches_to_optimize]) / 2
        cy.data[patches_to_optimize] = y2.data[patches_to_optimize] - length.data[patches_to_optimize] * torch.sin(theta.data[patches_to_optimize]) / 2
        constrain_parameters(cx, cy, theta, length, width, canvas_width=w, canvas_height=h, size_optimizer=size_optimizer)
        
        # 5. Snap lines that have coinciding ends, close orientations and close widths
        if (i + 1) % 20 == 0:
            snap_lines(cx, cy, theta, length, width, pos_optimizer=pos_optimizer, size_optimizer=size_optimizer)
    
    except KeyboardInterrupt:
        its_time_to_stop[0] = True
    
    # 6. Prepare the results for logging
    sigint = signal.signal(signal.SIGINT, plotting_sigint)
    
    if (i % 20 == 0) or its_time_to_stop[0]:
        # 6.1. Record the current values of parameters separately
        #      The following steps are performed with these separate values and do not affect the parameters being optimized
        cx_final[patches_to_optimize] = cx.data[patches_to_optimize]
        cy_final[patches_to_optimize] = cy.data[patches_to_optimize]
        theta_final[patches_to_optimize] = theta.data[patches_to_optimize]
        length_final[patches_to_optimize] = length.data[patches_to_optimize]
        width_final[patches_to_optimize] = width.data[patches_to_optimize]

        # 6.2. Collapse invisible lines completely
        width_final[width_final < 1/4] = 0
        
        # 6.3. Collapse the lines that don't add new information to the rasterization
        collapse_redundant_lines(cx_final, cy_final, theta_final, length_final, width_final, patches_to_consider=patches_to_optimize)
        x2 = cx_final.data[patches_to_optimize] + length_final.data[patches_to_optimize] * torch.cos(theta_final.data[patches_to_optimize]) / 2
        x1 = cx_final.data[patches_to_optimize] - length_final.data[patches_to_optimize] * torch.cos(theta_final.data[patches_to_optimize]) / 2
        y2 = cy_final.data[patches_to_optimize] + length_final.data[patches_to_optimize] * torch.sin(theta_final.data[patches_to_optimize]) / 2
        y1 = cy_final.data[patches_to_optimize] - length_final.data[patches_to_optimize] * torch.sin(theta_final.data[patches_to_optimize]) / 2
        lines_batch_final[patches_to_optimize] = torch.stack([x1, y1, x2, y2, width_final[patches_to_optimize]], -1)

        # 6.4. Render the lines and calculate the difference from the raster
        vector_rendering[patches_to_optimize] = render_lines_pt(lines_batch_final[patches_to_optimize])
        im = rasters_batch[patches_to_optimize] - vector_rendering[patches_to_optimize]
        # 6.5. Optionally, display rendered vector, difference, skeleton of vector
        if plotting:
            im_refined[patches_to_optimize] = vector_rendering[patches_to_optimize].cpu().numpy()
            im_dif[patches_to_optimize] = im.cpu().numpy()
            im_refined_skeleton[patches_to_optimize] = render_lines_skeleton(lines_batch_final[patches_to_optimize])
            refined_plot.set_array(np.vstack(im_refined))
            dif_plot.set_array(np.vstack(im_dif))
            refined_skeleton_plot.set_array(np.vstack(im_refined_skeleton))
            clear_output(wait=True)
            display(fig)
            # Optionally, save the images for further export
            ims.append([im_refined.copy(), im_dif.copy(), im_refined_skeleton.copy()])

        # 6.6. Optionally, calculate the current mean IOU score and plot
        if log_plotting:
            log_data.append(my_iou_score(vector_rendering, rasters_batch, average='mean'))
            log_data_x.append(i)
            clear_output(wait=True)
            log_plot_ax.set_xlim(0, i)
            log_plot.set_xdata(log_data_x)
            log_plot.set_ydata(log_data)
            print(log_data[-1])
            display(fig)

        
        # 7. Find the patches with high enough IOU value and do not optimize them furher
        #if i % 1 == 0:
        #    assert lines_batch[torch.isnan(lines_batch)].numel() == 0
        #    new_patches_to_optimize = my_iou_score(vector_rendering[patches_to_optimize], rasters_batch[patches_to_optimize]) < .98
        #    patches_to_optimize_left = new_patches_to_optimize.sum()
        #    print(patches_to_optimize_left)
        #    if patches_to_optimize_left == 0:
        #        break
        #    patches_to_optimize[patches_to_optimize] &= new_patches_to_optimize
        #    # stop optimizers momentum
        #    size_optimizer.state[length]['exp_avg'][~patches_to_optimize] = 0
        #    size_optimizer.state[width]['exp_avg'][~patches_to_optimize] = 0
        #    pos_optimizer.state[cx]['exp_avg'][~patches_to_optimize] = 0
        #    pos_optimizer.state[cy]['exp_avg'][~patches_to_optimize] = 0
        #    pos_optimizer.state[theta]['exp_avg'][~patches_to_optimize] = 0
    
    signal.signal(signal.SIGINT, sigint)
    if its_time_to_stop[0]:
        break

In [ ]:
plt.plot(log_data_x, log_data)

### Inspect results

In [ ]:
patch_id = 6

im_initial_pred = render_lines_pt(torch.from_numpy(initial_vector[patch_id][None]).type(dtype).to(device)).cpu().numpy()
im_refined = render_lines_pt(lines_batch_final[patch_id][None].detach()).cpu().numpy()
im_gt = rasters_batch[patch_id][None].cpu().numpy()
im_dif = im_refined - im_gt
im_initial_skeleton = render_lines_skeleton(torch.from_numpy(initial_vector[patch_id][None]))
im_refined_skeleton = render_lines_skeleton(lines_batch_final[patch_id][None].detach())

fig, [initial_pred_ax, initial_skeleton_ax, refined_ax, refined_skeleton_ax, gt_ax, dif_ax] = plt.subplots(1, 6, figsize=[4 * 6, 4 * 1])

initial_pred_plot = initial_pred_ax.imshow(np.vstack(im_initial_pred), vmin=0, vmax=1, cmap='gray_r')
refined_plot = refined_ax.imshow(np.vstack(im_refined), vmin=0, vmax=1, cmap='gray_r')
gt_plot = gt_ax.imshow(np.vstack(im_gt), vmin=0, vmax=1, cmap='gray_r')
dif_plot = dif_ax.imshow(np.vstack(im_dif), vmin=-1, vmax=1, cmap='gray_r')
initial_skeleton_plot = initial_skeleton_ax.imshow(np.vstack(im_initial_skeleton))
refined_skeleton_plot = refined_skeleton_ax.imshow(np.vstack(im_refined_skeleton))

In [ ]:
fig, axes = plt.subplots(2, 5, figsize=[4 * 5, 4 * 2])
for i, ax in enumerate(axes.reshape(-1)):
    ax.imshow(render_lines_pt(lines_batch_final[patch_id, i][None, None].detach()).cpu().numpy()[0], cmap='gray_r')

In [ ]:
ious = my_iou_score(vector_rendering, rasters_batch)
patches_to_draw = np.argsort(ious)[:20]
#patches_to_draw = np.random.choice(len(rasters_batch), 20, replace=False)
print(patches_to_draw)

im_initial_pred = render_lines_pt(torch.from_numpy(initial_vector[patches_to_draw]).type(dtype).to(device)).cpu().numpy()
im_refined = vector_rendering[patches_to_draw].cpu().numpy()
im_gt = rasters_batch[patches_to_draw].cpu().numpy()
im_dif = im_refined - im_gt
im_initial_skeleton = render_lines_skeleton(torch.from_numpy(initial_vector[patches_to_draw]))
im_refined_skeleton = render_lines_skeleton(lines_batch_final[patches_to_draw].detach())

fig, [initial_pred_ax, initial_skeleton_ax, refined_ax, refined_skeleton_ax, gt_ax, dif_ax] = plt.subplots(1, 6, figsize=[4 * 6, 4 * 20])

initial_pred_plot = initial_pred_ax.imshow(np.vstack(im_initial_pred), vmin=0, vmax=1, cmap='gray_r')
refined_plot = refined_ax.imshow(np.vstack(im_refined), vmin=0, vmax=1, cmap='gray_r')
gt_plot = gt_ax.imshow(np.vstack(im_gt), vmin=0, vmax=1, cmap='gray_r')
dif_plot = dif_ax.imshow(np.vstack(im_dif), vmin=-1, vmax=1, cmap='gray_r')
initial_skeleton_plot = initial_skeleton_ax.imshow(np.vstack(im_initial_skeleton))
refined_skeleton_plot = refined_skeleton_ax.imshow(np.vstack(im_refined_skeleton))

#### Export results